In [4]:
import tensorflow as tf
import numpy as np 
import math as m

In [5]:
class gru_model(tf.keras.Model):
    def __init__(self, rnn_units, num_layers):
        super(gru_model, self).__init__()
        self.num_layers = num_layers
        self.rnn_units = rnn_units
        self.grus = [tf.keras.layers.GRU(rnn_units[i],
                                   return_sequences=True, 
                                   return_state=True) for i in range(num_layers)]
        
        self.w1 = tf.keras.layers.Dense(10) ## Dense is a linear layer 
        self.w2 = tf.keras.layers.Dense(1) 


    def call(self, inputs, training=False):
        #input shape: (batch_size, seq_length, features)
        assert self.num_layers == len(self.rnn_units)
        x = inputs
        if (len(x.shape) != 3):
            print('what are you doing? this is wrong shape')
        print('input dim ({}, {}, {})'.format(x.shape[0], x.shape[1], x.shape[2]))
        whole_seq = x

        ## RNN stuff
        for i in range(self.num_layers):
            whole_seq, final_s = self.grus[i](whole_seq, training=training)
        ##
        
        ## extra layers
        target = self.w1(final_s)
        target = tf.nn.relu(target) ## just non-linearity 
        target = self.w2(target)
        
        ### The Dense layer right before the sigmoid must be dimension 1. 
        ### We must keep sigmoid for probability in [0, 1]
        target = tf.nn.sigmoid(target)
        
        return target

In [25]:
## if inputs are [x1, x2]
## Dense(x1, x2) == w1x1 + w2x2 

In [6]:
def nll(y, y_pred):
    '''y_pred is a probability value between 0 to 1'''
    bce = tf.keras.losses.BinaryCrossentropy()    
    return bce(y, y_pred)


In [7]:
def build_graph():
    
    @tf.function(experimental_relax_shapes=True)
    def train_step(gru, optimizer, x,y, training=True):

        with tf.GradientTape(persistent=True) as tape:

            μ = gru(x, training) 
            xent = nll(y, μ)
        
        gradients = tape.gradient(xent, gru.trainable_variables)
        optimizer.apply_gradients(zip(gradients, gru.trainable_variables))
        return μ, xent

    tf.keras.backend.set_floatx('float32')
    return train_step

In [9]:
y = np.random.choice([0, 1], size = 32)
y

array([1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0])

In [10]:
array = np.random.normal(size = (32, 20, 1))

In [11]:
array.shape # batch_size X seq_len (timestamps in time-series) X features

(32, 20, 1)

In [12]:
g_model = gru_model([32], 1) #term in [] is size of hidden layers

2023-05-09 22:03:29.165404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
g_model(array)
# output shape batch_size X 1 
# outptut is a probability between 0 to 1 
# interpretation is how likely each sequence to be a coral/not coral  

input dim (32, 20, 1)


<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[0.5095421 ],
       [0.4956858 ],
       [0.47816133],
       [0.4688766 ],
       [0.46282226],
       [0.46647865],
       [0.4871919 ],
       [0.48821723],
       [0.45809904],
       [0.47964028],
       [0.46525875],
       [0.47405523],
       [0.47509232],
       [0.44130936],
       [0.4762384 ],
       [0.47616076],
       [0.4909564 ],
       [0.4910372 ],
       [0.48490867],
       [0.4758533 ],
       [0.4857951 ],
       [0.48292905],
       [0.4855606 ],
       [0.52518517],
       [0.4895521 ],
       [0.4754499 ],
       [0.48418042],
       [0.48313034],
       [0.48713794],
       [0.44082937],
       [0.47536755],
       [0.4943494 ]], dtype=float32)>

In [14]:
nll(y, g_model(array))

input dim (32, 20, 1)


<tf.Tensor: shape=(), dtype=float32, numpy=0.68439996>

In [15]:
optimizer = tf.keras.optimizers.Adam(3e-4)

In [17]:
with tf.device("/CPU:0"):
    num_epochs = 1
    num_batches = 100
    tr_step = build_graph()
    for epoch in range(num_epochs):
        for batch in range(num_batches):
            array = np.random.normal(size = (32, 20, 1))# fake data
            y = np.random.choice([0, 1], size = 32)  # fake             
            ### array, y  = batcher_fun(data, training = True) shapes: (batch_s, seq_l, features), (batch_s, 1)
            y_pred, xent = tr_step(g_model, optimizer, array, y, training=True)
            
         ## validation set 
         ## test_set 

input dim (32, 20, 1)
